## The Original EigenDecompression.EigenDecompose

See

* https://mobile.twitter.com/realize_ss/status/1615160291108745216
* https://qiita.com/lelele/items/8408410a94f5c6b8f76e

In [1]:
using LinearAlgebra
M = rand(100,100)#対角化したい行列
E, P = eigen(M)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
100-element Vector{ComplexF64}:
 -2.9751216540721384 - 0.7361270868451844im
 -2.9751216540721384 + 0.7361270868451844im
 -2.8366486254651258 + 0.0im
 -2.4484641846954807 - 1.2159078705290065im
 -2.4484641846954807 + 1.2159078705290065im
  -2.383557536265102 - 0.6473616193805896im
  -2.383557536265102 + 0.6473616193805896im
 -2.0893584505639122 + 0.0im
 -2.0342009560006638 - 1.8457021937288598im
 -2.0342009560006638 + 1.8457021937288598im
 -1.9189352293259374 - 1.0826785867521127im
 -1.9189352293259374 + 1.0826785867521127im
  -1.839740625871089 + 0.0im
                     ⋮
  1.9612660936640003 + 0.31609722819482927im
  2.1721178114005077 - 1.3257556752057784im
  2.1721178114005077 + 1.3257556752057784im
  2.2880569126025625 - 0.2029150387620231im
  2.2880569126025625 + 0.2029150387620231im
   2.342227699415863 - 2.1638904456844994im
   2.342227699415863 + 2.1638904456844994im
  2.4414741909795317 - 0.653677

In [2]:
exp(eigen(M))

LoadError: MethodError: no method matching exp(::Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}})

[0mClosest candidates are:
[0m  exp([91m::Union{Float16, Float32, Float64}[39m)
[0m[90m   @[39m [90mBase[39m [90mspecial\[39m[90m[4mexp.jl:325[24m[39m
[0m  exp([91m::Adjoint{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:595[24m[39m
[0m  exp([91m::Transpose{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:596[24m[39m
[0m  ...


In [3]:
module EigenDecompression

export EigenDecompose, eigDecomp
using LinearAlgebra
import Base.*, Base./

#対角化された行列型
struct EigenDecompose{T<:Number} <: AbstractMatrix{T}
    P::AbstractMatrix{T}
    D::Diagonal{T}
    invP::AbstractMatrix{T}
end

#普通のMatrixを対角化する
function eigDecomp(mat::AbstractMatrix)
    E, P = eigen(mat)
    EigenDecompose(P, Diagonal(E), inv(P))
end

#EigenDecompose型に対する関数
Base.exp(eig::EigenDecompose) = EigenDecompose(eig.P, exp(eig.D), eig.invP)
*(eig::EigenDecompose, vec::AbstractVector) = eig.P * eig.D * eig.invP * vec
*(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D*sc, eig.invP)
/(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D/sc, eig.invP)

#普通のMatrixに戻す
Base.Array(eig::EigenDecompose) = eig.P * eig.D * eig.invP

end

Main.EigenDecompression

In [4]:
using .EigenDecompression

M = rand(100, 100)
eM = eigDecomp(M)
for i in 1:100
    v = rand(100)
    rnd = rand()
    @assert exp(M*rnd)*v ≈ exp(eM*rnd)*v
end

In [5]:
using BenchmarkTools

M = rand(100, 100);

#普通な方
function bench1(M)
    for i in 1:100
        v = rand(100)
        exp(M*rand())*v
    end
end

#今回実装した方
function bench2(M)
    eM = eigDecomp(M)
    for i in 1:100
        v = rand(100)
        exp(eM*rand())*v
    end
end

bench2 (generic function with 1 method)

In [6]:
@benchmark bench1(M)

BenchmarkTools.Trial: 66 samples with 1 evaluation.
 Range (min … max):  68.357 ms … 88.674 ms  ┊ GC (min … max): 2.19% … 8.10%
 Time  (median):     75.069 ms              ┊ GC (median):    2.24%
 Time  (mean ± σ):   75.864 ms ±  4.577 ms  ┊ GC (mean ± σ):  3.20% ± 1.38%

      ▂        ▅    ▅ █ █                                      
  ▅▅▅▅█▁▅▅▁▅▅████▅███████▅▅▅▁▁▅██▁▁▁▁▅▅█▅▁▅▁██▁▁▅▁▁▁▁▅▁▁▁▁▁▁▅ ▁
  68.4 ms         Histogram: frequency by time        88.1 ms <

 Memory estimate: 53.80 MiB, allocs estimate: 1900.

In [7]:
@benchmark bench2(M)

BenchmarkTools.Trial: 768 samples with 1 evaluation.
 Range (min … max):  5.800 ms …   8.036 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.476 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.512 ms ± 228.915 μs  ┊ GC (mean ± σ):  0.40% ± 1.82%

                        ▁▂▅▆██▇▆▄▃▂  ▁▁ ▁▁                     
  ▄▁▄▄▅▇█▅▆▅▆▅▄▁▄▆▅▆▁▅█▇███████████████▇████▇▆▆▅▁▄▄▅▆▇▅▇▅▇▁▆▅ █
  5.8 ms       Histogram: log(frequency) by time      7.24 ms <

 Memory estimate: 1.82 MiB, allocs estimate: 1728.

## EigenDecomposedMatrices.EigenDecomposed

In [8]:
module EigenDecomposedMatrices

export EigenDecomposed

using LinearAlgebra

struct EigenDecomposed{
        T,
        TA<:AbstractMatrix,
        TE<:AbstractVector,
        TP<:AbstractMatrix,
        TinvP<:AbstractMatrix
    } <: AbstractMatrix{T}
    A::TA
    E::TE
    P::TP
    invP::TinvP
end

function EigenDecomposed(A::AbstractMatrix, E::AbstractVector, P::AbstractMatrix, invP::AbstractMatrix)
    EigenDecomposed{eltype(A), typeof(A), typeof(E), typeof(P), typeof(invP)}(A, E, P, invP)
end

function EigenDecomposed(M::AbstractMatrix)
    E, P = eigen(M)
    A = oftype(P, M)
    invP = ishermitian(M) ? P' : inv(P)
    EigenDecomposed(A, E, P, invP)
end

Base.size(ed::EigenDecomposed) = size(ed.A)
Base.getindex(ed::EigenDecomposed, I...) = getindex(ed.A, I...)
Base.convert(::Type{Array}, ed::EigenDecomposed) = ed.A

function exp_old(ed::EigenDecomposed)
    (; A, E, P, invP) = ed
    expE = exp.(E)
    expA = P * Diagonal(expE) * invP
    EigenDecomposed(expA, expE, P, invP)
end
Base.exp(ed::EigenDecomposed) = exp_eigendecomposed(ed)
Base.:*(c::Number, ed::EigenDecomposed) = EigenDecomposed(c*ed.A, c*ed.E, ed.P, ed.invP)
Base.:*(ed::EigenDecomposed, c::Number) = EigenDecomposed(ed.A*c, ed.E*c, ed.P, ed.invP)
Base.:\(c::Number, ed::EigenDecomposed) = EigenDecomposed(c\ed.A, c\ed.E, ed.P, ed.invP)
Base.:/(ed::EigenDecomposed, c::Number) = EigenDecomposed(ed.A/c, ed.E/c, ed.P, ed.invP)
Base.:*(ed::EigenDecomposed, v::AbstractVector) = ed.A * v

"""
    exp_eigendecomposed!(Y, ed::EigenDecomposed, expE=similar(ed.E), tmpY=similar(Y))

returns the exponential of `ed` and stores the result in `Y`, overwriting the existing value of `Y`. 
It does not overwrite `ed` and uses `expE` and `tmpY` as workspaces.
"""
function exp_eigendecomposed!(Y, ed::EigenDecomposed, expE=similar(ed.E), tmpY=similar(Y))
    (; A, E, P, invP) = ed
    @. expE = exp.(E)
    mul!(tmpY, P, Diagonal(expE))
    mul!(Y, tmpY, invP)
end
exp_eigendecomposed(ed::EigenDecomposed) = exp_eigendecomposed!(similar(ed.P), ed)
LinearAlgebra.lmul!(c::Number, ed::EigenDecomposed) = (lmul!(c, ed.A); lmul!(c, ed.E))
LinearAlgebra.rmul!(ed::EigenDecomposed, c::Number) = (rmul!(ed.A, c); rmul!(ed.E, c))
LinearAlgebra.ldiv!(c::Number, ed::EigenDecomposed) = (ldiv!(c, ed.A); ldiv!(c, ed.E))
LinearAlgebra.rdiv!(ed::EigenDecomposed, c::Number) = (rdiv!(ed.A, c); rdiv!(ed.E, c))

for T in (AbstractVector, AbstractMatrix)
    @eval function LinearAlgebra.mul!(y::$T, ed::EigenDecomposed, x::$T, alpha::Number, beta::Number)
        mul!(y, ed.A, x, alpha, beta)
    end
end

end

Main.EigenDecomposedMatrices

In [9]:
?EigenDecomposedMatrices.exp_eigendecomposed!

```
exp_eigendecomposed!(Y, ed::EigenDecomposed, expE=similar(ed.E), tmpY=similar(Y))
```

returns the exponential of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `expE` and `tmpY` as workspaces.


In [10]:
methods(EigenDecomposedMatrices.EigenDecomposed)

# 2 methods for type constructor:
 [1] Main.EigenDecomposedMatrices.EigenDecomposed(M::AbstractMatrix)
     @ In[8]:24
 [2] Main.EigenDecomposedMatrices.EigenDecomposed(A::AbstractMatrix, E::AbstractVector, P::AbstractMatrix, invP::AbstractMatrix)
     @ In[8]:20

In [11]:
methodswith(EigenDecomposedMatrices.EigenDecomposed)

[1] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[8]:43
[2] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractVector) @ Main.EigenDecomposedMatrices In[8]:46
[3] *(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[8]:42
[4] /(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[8]:45
[5] \(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[8]:44
[6] convert(::Type{Array}, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[8]:33
[7] exp(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[8]:41
[8] getindex(ed::Main.EigenDecomposedMatrices.EigenDecomposed, I...) @ Main.EigenDecomposedMatrices In[8]:32
[9] size(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[8]:31
[10] ldiv!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[8]:63
[11] lmul!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[8]:61
[12] mul!(y::AbstractVector, ed::Main.EigenDecomposedMatrices.EigenDecomposed, x::AbstractVector, alpha::Number, beta::Number) @ Main.EigenDecomposedMatrices In[8]:67
[13] mul!(y::AbstractMatrix, ed::Main.EigenDecomposedMatrices.EigenDecomposed, x::AbstractMatrix, alpha::Number, beta::Number) @ Main.EigenDecomposedMatrices In[8]:67
[14] rdiv!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[8]:64
[15] rmul!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[8]:62

In [12]:
methods(EigenDecomposedMatrices.exp_eigendecomposed!)

# 3 methods for generic function "exp_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[8]:54
 [2] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE)
     @ In[8]:54
 [3] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE, tmpY)
     @ In[8]:54

In [13]:
n = 2^8
M = 5I + randn(n, n)
v = randn(n)
c = randn()

edM = EigenDecomposedMatrices.EigenDecomposed(M)

Y = similar(edM.A)
expE = similar(edM.E)
tmpY = similar(Y)

y = similar(v)
tmpy = oftype(edM.E, y)
alpha = randn()
beta = randn();

In [14]:
edM

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}:
   5.65324+0.0im   0.811664+0.0im  …  -0.859246+0.0im   -0.867373+0.0im
  -1.20315+0.0im    6.35092+0.0im       0.16714+0.0im    0.281514+0.0im
  0.229875+0.0im  -0.453096+0.0im      0.768638+0.0im   -0.433825+0.0im
  0.742781+0.0im   -1.28684+0.0im     -0.586964+0.0im    -1.79043+0.0im
  0.609952+0.0im  -0.808625+0.0im      0.567903+0.0im  -0.0417683+0.0im
 -0.756712+0.0im   -1.73971+0.0im  …  -0.895302+0.0im   -0.727094+0.0im
 -0.942006+0.0im  -0.695088+0.0im     0.0502155+0.0im    0.834619+0.0im
 -0.721786+0.0im  -0.135759+0.0im      0.421936+0.0im    0.717119+0.0im
 -0.343408+0.0im    0.63547+0.0im     -0.604267+0.0im     1.13966+0.0im
  -0.36897+0.0im   0.723486+0.0im      0.306799+0.0im     1.94141+0.0im
  0.400205+0.0im   -1.03729+0.0im  …  0.0994644+0.0im    0.565096+0.0im
  -1.12538+0.0im  -0.732021+0.0im       1.64388+0.0im   -0.278

In [15]:
dump(edM)

Main.EigenDecomposedMatrices.EigenDecomposed{ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}
  A: Array{ComplexF64}((256, 256)) ComplexF64[5.653240359790474 + 0.0im 0.8116635437078049 + 0.0im … -0.8592461147165348 + 0.0im -0.8673734639992252 + 0.0im; -1.2031483417318523 + 0.0im 6.350920262183919 + 0.0im … 0.16713954681326654 + 0.0im 0.28151448021470293 + 0.0im; … ; 0.4720489945034345 + 0.0im -0.6452920480150005 + 0.0im … 6.11287275336207 + 0.0im -0.09775410755150953 + 0.0im; 0.6014206821441552 + 0.0im 1.076439839583611 + 0.0im … -0.4578902312844247 + 0.0im 4.188209737806721 + 0.0im]
  E: Array{ComplexF64}((256,)) ComplexF64[-11.353090158082509 + 0.0im, -10.32210154913398 - 1.861974982038516im, -10.32210154913398 + 1.861974982038516im, -10.297200076170467 - 5.368797892732106im, -10.297200076170467 + 5.368797892732106im, -9.056997191260702 - 4.030008990882876im, -9.056997191260702 + 4.030008990882876im, -8.754271496477859 - 1.4600949380533892im

In [16]:
M ≈ Matrix(edM)

true

In [17]:
M ≈ edM

true

In [18]:
c*M ≈ c*edM ≈ edM*c

true

In [19]:
c\M ≈ c\edM ≈ edM/c

true

In [20]:
(
    exp(M)
    ≈ exp(edM)
    ≈ EigenDecomposedMatrices.exp_old(edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY)
)

true

In [21]:
@show typeof(y)
mul!(y, M, v, alpha, beta) ≈ mul!(y, edM, v, alpha, beta)

typeof(y) = Vector{Float64}


true

In [22]:
@show typeof(tmpy)
(
    exp(M) * v
    ≈ exp(edM) * v
    ≈ EigenDecomposedMatrices.exp_old(edM) * v
    ≈ mul!(tmpy, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM), v)
    ≈ mul!(tmpy, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY), v)
)

typeof(tmpy) = Vector{ComplexF64}


true

In [23]:
@btime edM = EigenDecomposedMatrices.EigenDecomposed(M);

  35.788 ms (28 allocations: 4.53 MiB)


In [24]:
@btime exp($M) * $v
@btime exp($edM) * $v
@btime EigenDecomposedMatrices.exp_old($edM) * $v
@btime mul!($tmpy, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM), $v)
@btime mul!($tmpy, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM, $expE, $tmpY), $v);

  9.392 ms (16 allocations: 3.01 MiB)
  2.485 ms (7 allocations: 2.01 MiB)
  2.481 ms (7 allocations: 2.01 MiB)
  2.349 ms (3 allocations: 1.00 MiB)
  2.202 ms (0 allocations: 0 bytes)


In [25]:
n2 = 2^8
M2 = Symmetric(5I + randn(n2, n2))
v2 = randn(n)
c2 = randn()

edM2 = EigenDecomposedMatrices.EigenDecomposed(M2)

Y2 = similar(edM2.A)
expE2 = similar(edM2.E)
tmpY2 = similar(Y2)

y2 = similar(v2)
tmpy2 = oftype(edM2.E, y2)
alpha2 = randn()
beta2 = randn();

In [26]:
edM2

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Matrix{Float64}, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  3.6473      -0.00782139  -0.435994   …   2.05575    -0.249281    0.587851
 -0.00782139   5.28651      0.131905       0.695865   -1.63094     0.583392
 -0.435994     0.131905     5.21008       -0.775603   -0.81029    -0.0374679
 -0.0769022   -0.335312     0.648623      -0.739495    0.621381   -1.48564
 -1.17094      1.35226      0.0443991      0.0735607   0.0549492   0.419179
  1.02721      1.44404     -0.0222104  …   0.803171    0.336903    1.0392
 -1.39573     -0.809841    -0.78849       -0.0727124  -0.970087   -1.17958
  0.226949    -0.173972    -0.714227       0.43552     1.03035     0.859974
  0.558163     0.647521     0.833877       0.554956    1.85963     0.0859631
  0.762047    -0.408986     1.90391        1.48416     0.414748    0.0762793
 -0.344087     0.40267      1.12732    …   0.669697    0.187838    0.15391
 -0.794478     0.

In [27]:
dump(edM2)

Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Matrix{Float64}, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}
  A: Array{Float64}((256, 256)) [3.6473037451227297 -0.00782139350902675 … -0.24928079518286228 0.5878511392913932; -0.00782139350902675 5.286509064562285 … -1.630937812174819 0.5833916370789631; … ; -0.24928079518286228 -1.630937812174819 … 6.101596058446986 0.385368144948812; 0.5878511392913932 0.5833916370789631 … 0.385368144948812 3.889279373400584]
  E: Array{Float64}((256,)) [-26.076276490208027, -25.239068382181962, -24.939945334190398, -23.839257544987905, -23.5983515819698, -23.54900040977536, -23.102530353964465, -22.769248576893425, -22.492489063227726, -21.822935288841673  …  31.789738919208137, 32.16508641952741, 32.53493593530226, 32.717394738860534, 33.520275725483884, 33.651503888385676, 34.200240012037284, 34.67922559347302, 35.265230151313474, 36.74795337385183]
  P: Array{Float64}((256, 256)) [-0.0791786726633903 0.00899604009395

In [28]:
M2 ≈ Matrix(edM2)

true

In [29]:
M2 ≈ edM2

true

In [30]:
c2*M2 ≈ c2*edM2 ≈ edM2*c2

true

In [31]:
c2\M2 ≈ c2\edM2 ≈ edM2/c2

true

In [32]:
(
    exp(M2)
    ≈ exp(edM2)
    ≈ EigenDecomposedMatrices.exp_old(edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2)
)

true

In [33]:
@show typeof(y2)
mul!(y2, M2, v2, alpha2, beta2) ≈ mul!(y2, edM2, v2, alpha2, beta2)

typeof(y2) = Vector{Float64}


true

In [34]:
@show typeof(tmpy2)
(
    exp(M2) * v2
    ≈ exp(edM2) * v2
    ≈ EigenDecomposedMatrices.exp_old(edM2) * v2
    ≈ mul!(tmpy2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2), v2)
    ≈ mul!(tmpy2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2), v2)
)

typeof(tmpy2) = Vector{Float64}


true

In [35]:
@btime edM2 = EigenDecomposedMatrices.EigenDecomposed(M2);

  6.327 ms (16 allocations: 2.09 MiB)


In [36]:
@btime exp($M2) * $v2
@btime exp($edM2) * $v2
@btime EigenDecomposedMatrices.exp_old($edM2) * $v2
@btime mul!($tmpy2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2), $v2)
@btime mul!($tmpy2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2, $expE2, $tmpY2), $v2);

  6.853 ms (19 allocations: 2.60 MiB)
  639.400 μs (6 allocations: 1.00 MiB)
  629.900 μs (6 allocations: 1.00 MiB)
  637.000 μs (3 allocations: 514.17 KiB)
  597.100 μs (0 allocations: 0 bytes)
